# Data Loading 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 2.6 MB 30.6 MB/s 
     |████████████████████████████████| 1.2 MB 44.0 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 636 kB 60.5 MB/s 
     |████████████████████████████████| 895 kB 63.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=8b1a616a1ee8cd86c729f660125d7ea0fecb5739814d984c05531934a0add3fa
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics import silhouette_samples, silhouette_score , cohen_kappa_score
import gensim
from sentence_transformers import SentenceTransformer, util
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
from sklearn import svm

In [ ]:
train_df= pd.read_csv("/content/drive/MyDrive/DEBI/Ottawa/Data Science Applications/Assignments/Group_project/final_project/Dataset/train.csv")
test_df= pd.read_csv("/content/drive/MyDrive/DEBI/Ottawa/Data Science Applications/Assignments/Group_project/final_project/Dataset/test.csv")
test_labels_df= pd.read_csv("/content/drive/MyDrive/DEBI/Ottawa/Data Science Applications/Assignments/Group_project/final_project/Dataset/sample_submission.csv")

In [ ]:
train_df= pd.read_csv("/content/drive/MyDrive/Group_project/final_project/Dataset/training_data_cleaned.csv")

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Functions

In [ ]:
##Elbow method for clustering
def optimum_k(max_k,X):
  wcss = []
  for i in range(2, max_k):
    #kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans =MiniBatchKMeans(n_clusters=i, init='k-means++', n_init=1, init_size=1000, batch_size=1000, verbose=False, max_iter=1000)
    cluster_labels=kmeans.fit(X)
    print("model finished training")
    silhouette_avg = silhouette_score(X, kmeans.fit_predict(X))
    print("For n_clusters =", i,
          "The average silhouette_score is :", silhouette_avg)
    wcss.append(kmeans.inertia_)
  plt.plot(range(2, max_k), wcss)
  plt.title('Elbow Method')
  plt.xlabel('Number of clusters')
  plt.ylabel('WCSS')
  plt.show()
#return wcss

In [ ]:
##getting length of each question
def ques_len(train_df):
    q=train_df.split(" ")
    return len(q)

In [ ]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))

In [ ]:
##Getting the avergae vector of the all the word embeddings in a sentence
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

   
def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)



In [ ]:
# Method contains all the classification algorithms
def estimators(features, labels, estimator):
  X_train, X_test, y_train, y_test = train_test_split(
     features, labels, test_size=0.2, random_state=0)
  if (estimator == 'SVM'):
    model = svm.SVC().fit(X_train, y_train )
    text = 'SVM'
  if (estimator == 'DecisionTree'):
    model = DecisionTreeClassifier(max_depth=10).fit(X_train, y_train)
    text = 'Decision Tree'
  if (estimator == 'KNN'):
    model = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)
    text = 'KNN'
  if (estimator == 'RandomForest'):
    text = 'RandomForest'
    model=RandomForestClassifier(max_depth=15, random_state=0).fit(X_train, y_train)

  train_prediction = model.predict(X_train)
  prediction = model.predict(X_test)
  print(text, " Train Accuracy : ", accuracy_score(y_train,train_prediction)*100)
  print(text, " Test Accuracy : ", accuracy_score(y_test,prediction)*100)
  print("\n\t\tTEST DATA METRICS")
  print(text, " Confusion Matrix: ",confusion_matrix(y_test, prediction))
  print(text, " Report : ")
  print(classification_report(y_test,prediction))  


# Feature Engineering

Word2VEC

In [ ]:
##Getting all unique questions in the dataset
all_questions=pd.concat([train_df['question1'] , train_df['question2']],axis=0, ignore_index=True)
all_questions=all_questions.unique()

In [ ]:
##Transform all the training questions
documents=[]
for q in all_questions:
    documents.append(gensim.utils.simple_preprocess(str(q).encode('utf-8')))

In [ ]:
len(documents)

536013

In [ ]:
###Buiding the Word2vec model
model = gensim.models.Word2Vec(size=300, window=10, min_count=10, sg=1, workers=10)
model.build_vocab(documents)  

In [ ]:
###Training the Word2vec model
model.train(sentences=documents, total_examples=len(documents), epochs=model.iter)
model.corpus_count

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


536013

In [ ]:
##Transform the 2 questions in the train dataframe
q1 = []
q2=[]
for index, row in train_df.iterrows():
    q1.append(read_questions(row,"question1"))
    q2.append(read_questions(row,"question2"))

In [ ]:
###Gettinhg the feature vector of the first question
w2v_q1 = averaged_word_vectorizer(corpus=q1, model=model,num_features=300)
w2v_q1=pd.DataFrame(w2v_q1)
w2v_q1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.002163,0.035122,0.035819,0.080290,-0.127455,-0.195097,0.129632,0.012980,0.017270,-0.139771,-0.052276,-0.015170,0.026436,-0.046106,0.014357,-0.172406,0.172847,-0.031739,0.024736,-0.036092,-0.251780,-0.137283,0.043327,0.209316,-0.090279,-0.094340,0.037664,-0.020217,0.094674,-0.140906,-0.147094,-0.059406,-0.145525,0.069240,0.106495,-0.192783,0.180365,0.007296,0.043680,0.102318,...,0.136424,0.056346,0.070465,-0.074184,-0.078636,0.013977,-0.053577,0.084379,0.098539,0.020164,0.010111,-0.003979,-0.021690,0.025680,-0.037753,0.181907,-0.058441,-0.028937,0.120210,0.031621,0.038254,0.038463,0.020142,-0.096275,0.140266,0.077175,-0.040723,0.075920,-0.190359,-0.019146,-0.062460,0.109519,-0.057846,0.008718,-0.057391,0.073275,-0.044224,0.080690,0.018476,0.099836
1,0.125517,0.040885,0.002634,0.027717,-0.097512,-0.260680,0.061009,0.051703,0.035252,-0.122280,0.003330,0.015283,0.020666,-0.048231,-0.055681,-0.075096,0.075293,-0.000424,0.061264,-0.103155,-0.221954,-0.045125,-0.069526,0.176508,-0.091669,0.020900,-0.020063,0.043110,0.095051,-0.177319,-0.106537,-0.044107,0.031159,0.107986,0.164050,-0.200838,0.181508,0.093262,-0.009593,0.073111,...,0.057935,0.059165,0.006636,-0.037180,-0.087129,-0.037673,-0.083051,0.055666,0.016815,0.002815,0.128640,-0.002452,0.027006,-0.017187,-0.062249,0.040854,0.008765,-0.103796,0.000034,0.122168,0.130311,-0.064587,-0.030344,-0.120849,0.135307,-0.003176,0.019094,0.039309,-0.173098,-0.004068,0.064552,0.092786,-0.074986,-0.092073,0.124897,0.121293,-0.071283,0.113860,0.037437,-0.029801
2,0.027636,0.036000,0.000284,0.084629,-0.170868,-0.072945,0.191756,0.128479,-0.116690,0.128140,-0.051011,-0.092370,-0.020107,-0.152984,0.204748,-0.161526,0.049819,0.019887,0.122460,-0.155580,-0.184943,0.022622,0.092670,0.201726,-0.046593,-0.041185,0.025331,0.167134,0.078739,-0.098673,-0.176630,-0.158972,-0.013273,0.165712,0.152212,-0.132907,0.136436,0.181070,0.153263,0.075403,...,0.000243,0.064265,-0.144807,0.009674,0.004838,-0.054514,-0.033621,0.053500,-0.092143,-0.126329,0.023916,0.044758,0.025657,-0.109099,0.067679,0.137411,-0.046691,0.033492,-0.016260,0.031983,0.020691,-0.067457,-0.018843,-0.202349,0.176044,0.069759,0.086411,0.040028,-0.098950,-0.083675,-0.041226,0.009160,-0.094366,0.173614,-0.050696,0.009985,-0.144106,0.152622,-0.059203,0.070782
3,0.030151,-0.052437,-0.072858,-0.033903,-0.241715,-0.245293,-0.017979,0.056697,-0.055551,-0.028526,-0.038385,-0.161696,-0.055732,-0.200679,0.139876,-0.104797,0.010157,0.030354,0.038388,-0.120482,-0.089914,-0.006215,0.037878,0.259407,-0.258481,-0.063186,0.081307,0.105329,0.018137,0.010288,-0.128425,-0.280407,-0.145920,0.027172,0.069344,-0.054411,0.193812,0.054636,0.028974,0.150698,...,0.145036,0.030045,-0.092498,0.196459,-0.302595,-0.084082,-0.046537,0.093960,0.009082,-0.044578,0.000809,0.031871,0.099935,-0.126385,0.083117,0.090465,-0.090463,0.036027,-0.010596,-0.013625,-0.084403,0.028801,-0.103475,-0.160023,0.288690,-0.115082,0.062536,-0.078020,-0.189327,-0.063849,-0.124212,0.037154,-0.136418,0.081586,0.022412,0.044791,-0.109677,0.013263,-0.008422,0.044738
4,0.093965,0.158807,-0.077798,0.110590,-0.137603,-0.133700,-0.061604,0.128639,0.159119,-0.007342,0.089216,-0.031685,-0.079609,-0.119939,0.061430,-0.210301,-0.147206,0.027828,0.176474,-0.169055,-0.200878,0.025314,0.085715,0.066942,-0.156084,0.129519,-0.079695,-0.015821,0.046214,0.013741,-0.044095,-0.090653,0.058446,0.094161,0.202813,-0.174566,0.235988,-0.014625,0.196560,0.081944,...,0.030017,-0.110213,-0.088019,0.083300,-0.162359,0.050232,-0.079966,0.063440,0.029063,-0.136067,0.129312,-0.047939,0.026699,-0.096105,-0.023499,0.174090,-0.238289,-0.140972,-0.024266,-0.018695,0.181820,0.034405,-0.071015,-0.025482,0.367904,-0.147210,0.072322,-0.036324,-0.273361,0.011621,

In [ ]:
###Gettinhg the feature vector of the second question
w2v_q2 = averaged_word_vectorizer(corpus=q2, model=model,num_features=300)
w2v_q2=pd.DataFrame(w2v_q2)
w2v_q2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.006770,0.035375,0.044350,0.057829,-0.115635,-0.171626,0.112717,0.011007,0.029502,-0.122288,-0.074823,-0.010507,0.035184,-0.036754,0.033036,-0.171955,0.164561,-0.066252,-0.001549,-0.029853,-0.273331,-0.110427,0.025161,0.228067,-0.100394,-0.088381,-0.025435,-0.062923,0.074644,-0.155660,-0.141384,-0.072877,-0.179140,0.071767,0.121223,-0.210200,0.188826,0.025963,0.016460,0.126630,...,0.138471,0.051584,0.066754,-0.102597,-0.073217,0.041925,-0.040507,0.079605,0.102217,0.017842,-0.015037,0.007262,-0.008160,0.033637,-0.047274,0.176393,-0.048884,-0.026021,0.121512,0.026412,0.025708,0.063131,0.035221,-0.095442,0.122751,0.069848,-0.069591,0.074363,-0.163677,-0.001447,-0.071883,0.090034,-0.059738,0.021351,-0.039727,0.068792,-0.052270,0.099899,-0.013076,0.103618
1,0.025677,0.041063,-0.010478,0.125864,-0.153728,-0.237105,0.138084,0.109157,0.000687,-0.107416,-0.048372,-0.000957,0.004353,-0.046702,-0.073478,-0.166488,-0.038808,-0.014426,0.012046,-0.224775,-0.161864,-0.094914,-0.031436,0.076708,-0.109162,-0.017987,-0.020014,0.014498,0.103899,-0.064647,-0.159054,-0.119929,-0.128476,0.055531,0.138652,-0.195239,0.202705,-0.042660,-0.050869,0.011657,...,0.161259,0.136491,0.045777,0.039886,-0.192229,0.018224,-0.054691,-0.015451,0.091640,-0.028081,0.081818,0.128608,0.043925,0.018105,-0.018391,0.189950,0.030789,-0.045267,0.018710,0.043873,0.050564,-0.054618,-0.044662,-0.136470,0.178675,-0.015606,0.063794,0.003317,-0.223065,0.076823,-0.045094,0.143073,-0.006911,-0.058510,0.038092,0.099946,-0.053367,0.085994,0.113280,-0.021845
2,0.077759,-0.016381,0.077047,0.098422,-0.207392,-0.136691,0.210272,0.026431,-0.056066,0.128840,-0.033147,0.001187,-0.047874,-0.126005,0.217938,-0.150199,0.092783,-0.085469,0.165031,-0.204510,-0.169255,-0.075837,0.166701,0.263945,-0.135471,0.012720,0.006870,0.056879,0.095398,-0.130474,-0.212130,-0.136630,-0.018187,0.071787,0.159427,-0.098913,0.132770,0.122076,0.167503,0.044963,...,0.064910,0.051673,-0.187855,0.046922,-0.076063,-0.054391,-0.079231,-0.023488,-0.051817,-0.121312,0.015360,0.125372,0.056159,-0.036989,0.008194,0.089484,-0.034102,0.065161,0.107047,-0.000585,0.095759,-0.134772,0.001528,-0.124444,0.139608,0.108835,-0.075976,-0.030122,-0.197230,0.001449,-0.110807,-0.018022,-0.047666,0.190168,-0.001577,-0.035675,-0.144106,0.109393,-0.052015,0.087755
3,0.187805,0.145141,-0.122112,-0.057024,-0.174879,-0.341742,-0.006988,0.129127,0.032518,-0.193707,0.009471,-0.002763,-0.129950,-0.212242,0.210302,-0.043794,0.014973,0.032812,0.081041,-0.229818,-0.166019,0.089435,0.064523,0.386981,-0.145883,-0.061587,-0.055765,0.018053,-0.055151,-0.067727,-0.297031,-0.172887,-0.256486,0.113150,0.024793,-0.242646,0.219499,0.089579,0.092485,0.115697,...,0.052821,-0.125429,-0.164286,0.272388,-0.070644,-0.170021,-0.055791,0.131320,0.144192,0.081724,0.092085,-0.138915,0.082932,0.027455,-0.177780,-0.026713,0.021417,-0.071738,0.044241,0.117010,0.021830,-0.030250,-0.030706,-0.293017,0.251517,0.102950,-0.094357,-0.194424,-0.194024,0.013925,-0.024201,-0.069443,-0.089295,-0.058624,0.028572,0.083252,-0.136099,0.092510,0.118084,-0.092567
4,0.099501,0.202996,-0.092809,0.147664,-0.176063,-0.237572,0.075940,0.171197,0.108704,-0.036627,0.098429,0.005307,-0.106001,-0.083511,-0.021120,-0.176958,-0.127362,0.009651,0.165747,-0.137530,-0.182989,-0.092473,0.051795,0.085957,-0.094166,0.000128,0.018613,0.069603,0.034281,-0.119865,0.103738,-0.027658,0.011223,0.095228,0.129287,-0.066396,0.271292,-0.147394,0.173602,0.064841,...,0.034986,-0.041181,-0.105363,0.106510,-0.216116,0.054549,-0.124350,0.021950,-0.016887,-0.149604,0.056061,0.028158,0.044853,-0.166660,0.124430,0.381972,-0.107089,-0.241160,-0.020549,-0.088574,0.124439,0.042976,-0.046354,0.015025,0.403595,-0.074358,0.185899,0.051971,-0.248449,0

In [ ]:
##Getting absolute difference between the two questions 
x=abs(w2v_q1-w2v_q2)

In [ ]:
np.array(w2v_q2).shape

(404199, 300)

#Models

In [ ]:
print('Decision Tree on word2vec')
estimators(x[:200000], train_df.is_duplicate.values[:200000], 'DecisionTree')

Decision Tree on word2vec
Decision Tree  Train Accuracy :  64.42999999999999
Decision Tree  Test Accuracy :  62.395

		TEST DATA METRICS
Decision Tree  Confusion Matrix:  [[17038  8029]
 [ 7013  7920]]
Decision Tree  Report : 
              precision    recall  f1-score   support

           0       0.71      0.68      0.69     25067
           1       0.50      0.53      0.51     14933

    accuracy                           0.62     40000
   macro avg       0.60      0.61      0.60     40000
weighted avg       0.63      0.62      0.63     40000



In [ ]:
print('Random Forest on word2vec')
estimators(x[:200000], train_df.is_duplicate.values[:200000], 'RandomForest')

Random Forest on word2vec
RandomForest  Train Accuracy :  81.49187500000001
RandomForest  Test Accuracy :  66.8725

		TEST DATA METRICS
RandomForest  Confusion Matrix:  [[21551  3516]
 [ 9735  5198]]
RandomForest  Report : 
              precision    recall  f1-score   support

           0       0.69      0.86      0.76     25067
           1       0.60      0.35      0.44     14933

    accuracy                           0.67     40000
   macro avg       0.64      0.60      0.60     40000
weighted avg       0.65      0.67      0.64     40000

